In [2]:
import math
import random
import numpy as np

In [3]:
# sta_amount: Total contend stations number
# RA_RU: RA_RU number reserved for UORA
# Lmax: Retransmit limit
# TXOP: Transaction Opportunity (time length in ms)
# ocw_min: minimum OCW
# ocw_max: maximum OCW
# CR: Capture Ratio (CR=infinite = no capture, CR=0 = always capture)

def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, CR, step, gamma):
    total_power = 0
    counter_delay_slot=0
    success_sta=0
    # setting ocw for each transmission [ocw_min, 1st_retry_ocw...ocw_max]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax: Reserved slot for transmission (total slots in worst case)
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)

    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter, obo_counter[STA1_obo_counter, STA2_obo_counter...]
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time, retrans_counter[STA1_obo_counter, STA2_obo_counter...]
    retrans_counter = [0 for _ in range(0, sta_amount)]
    # STA's distance from AP (From 0~1)
    sta_distance = [random.uniform(0, 1) for _ in range(0, sta_amount)]
    # record STAs choosen RA-RU, sta_on_R[slot_number][STA_1, STA_26...]
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    # Accumulated station transmit power usage
    sta_power_usage = [0 for _ in range(0, sta_amount)]
    
    capture_power_ratio = 10 ** (CR / 10)

    
    # No.i slot    
    for i in range(0, Imax):
        # STA's Transmit power [1P, 2P, 4P, 8P]
#         sta_power = [2 ** random.randint(0, 3) for _ in range(0, sta_amount)]
        sta_power = [2 ** (x if x <= (step - 1) else (step - 1)) for x in retrans_counter]

        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < Lmax):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
                
                # record power usage
                sta_power_usage[x] += sta_power[x]
#                 sta_power_usage[x] += (2 ** retrans_counter[x])
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # Record power (no power control so always 1)
            for sta_index in sta_on_R[i][r]:
                total_power += sta_power[sta_index]
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record access delay slot number
                counter_delay_slot += (i+1)
                success_sta += 1
#                 total_power += sta_power_usage[success_sta_index]
            # 2 STA on the same R -> capture process
            elif (len(sta_on_R[i][r]) == 2):
                success_sta_index_1 = sta_on_R[i][r][0]
                success_sta_index_2 = sta_on_R[i][r][1]
                receive_power_1 = sta_power[success_sta_index_1] / (sta_distance[success_sta_index_1] ** gamma)
                receive_power_2 = sta_power[success_sta_index_2] / (sta_distance[success_sta_index_2] ** gamma)
                
                if (receive_power_1 / receive_power_2) > capture_power_ratio:
                    # Capture sta 1
                    obo_counter[success_sta_index_1] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
#                     total_power += sta_power_usage[success_sta_index_1]
                    
                    # fail STAs 2
                    retrans_counter[success_sta_index_2] += 1
                    if (retrans_counter[success_sta_index_2] < Lmax):
                        obo_counter[success_sta_index_2] = random.randint(0, ocw[retrans_counter[success_sta_index_2]])
                elif (receive_power_1 / receive_power_2) < (1 / capture_power_ratio):
                    # Capture sta 2
                    obo_counter[success_sta_index_2] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
#                     total_power += sta_power_usage[success_sta_index_2]
                    
                    # fail STA 1
                    retrans_counter[success_sta_index_1] += 1
                    if (retrans_counter[success_sta_index_1] < Lmax):
                        obo_counter[success_sta_index_1] = random.randint(0, ocw[retrans_counter[success_sta_index_1]])
                else:
                    retrans_counter[success_sta_index_1] += 1
                    if (retrans_counter[success_sta_index_1] < Lmax):
                        obo_counter[success_sta_index_1] = random.randint(0, ocw[retrans_counter[success_sta_index_1]])
                    retrans_counter[success_sta_index_2] += 1
                    if (retrans_counter[success_sta_index_2] < Lmax):
                        obo_counter[success_sta_index_2] = random.randint(0, ocw[retrans_counter[success_sta_index_2]])
                
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < Lmax):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])
    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    average_power = 0
    if success_sta > 0:
        average_power = total_power / success_sta
#     average_power = total_power / sta_amount
    
    return success_probability, access_delay, through_put, utilization, average_power

In [6]:
sample = 1000

# 1, 2, ... 10, 20, 30, ... 100, 200, 300, ... 1000
m_list_simu = np.concatenate([np.arange(1, 11, 1), np.arange(20, 110, 10), np.arange(200, 1100, 100)])
Ps_list_simu = [0 for i in range(28)]
Da_list_simu = [0 for i in range(28)]
T_list_simu = [0 for i in range(28)]
U_list_simu = [0 for i in range(28)]
power_list_simu = [0 for i in range(28)]

# m_list_simu = [(i+1)*10 for i in range(20)]
# Ps_list_simu = [0 for i in range(20)]
# Da_list_simu = [0 for i in range(20)]
# T_list_simu = [0 for i in range(20)]
# U_list_simu = [0 for i in range(20)]
# power_list_simu = [0 for i in range(20)]

# m_list_simu = [(i+1)*10 for i in range(50)]
# Ps_list_simu = [0 for i in range(50)]
# Da_list_simu = [0 for i in range(50)]
# T_list_simu = [0 for i in range(50)]
# U_list_simu = [0 for i in range(50)]
# power_list_simu = [0 for i in range(50)]

# R = 148
# CR = 1.5
step = 4
R_list = [9, 18, 37, 74, 148]
# R_list = [148]
# CR_list = [3]
CR_list = [1.5, 3, 6, float('inf')]
for CR in CR_list:
    print(f'------------ CR={CR}--------------')
    for R in R_list:
#         print(f'------------ R={R}--------------')
        for idx, m in enumerate(m_list_simu):
            success_probability_total = 0;
            access_delay_total = 0;
            through_put_total = 0;
            utilization_total = 0;
            no_success_counter = 0;
            power_usage_total = 0;
            for i in range(sample):
                success_probability, access_delay, through_put, utilization, power_usage = simulation(sta_amount=m, RA_RU=R, Lmax=5, TXOP=5.673, ocw_min=7, ocw_max=31, CR=CR, step=step, gamma=2)
                success_probability_total += success_probability
                access_delay_total += access_delay
                through_put_total += through_put
                utilization_total += utilization
                power_usage_total += power_usage
                if success_probability == 0:
                    no_success_counter += 1

            Ps = success_probability_total / sample
            if (sample - no_success_counter > 0):
                Da = access_delay_total / (sample - no_success_counter)
            T = through_put_total / sample
            U = utilization_total / sample
#             power = power_usage_total / (sample - no_success_counter)
            power = power_usage_total / sample


            Ps_list_simu[idx] = Ps
            Da_list_simu[idx] = Da
            T_list_simu[idx] = T
            U_list_simu[idx] = U
            power_list_simu[idx] = power

#         print(f'ps_list_simu_{R}_ramp = {Ps_list_simu}')
#         print(f'da_list_simu_{R}_ramp = {Da_list_simu}')
#         print(f't_list_simu_{R}_ramp = {T_list_simu}')
#         print(f'u_list_simu_{R}_ramp = {U_list_simu}')
#         print(f'power_list_simu_{R}_ramp = {power_list_simu}')
        print(f'ps_list_simu_{R} = {Ps_list_simu}')
        print(f'da_list_simu_{R} = {Da_list_simu}')
        print(f't_list_simu_{R} = {T_list_simu}')
        print(f'u_list_simu_{R} = {U_list_simu}')
        print(f'power_list_simu_{R} = {power_list_simu}')
        print("\n")

------------ CR=1.5--------------
ps_list_simu_9 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9997500000000001, 0.9906333333333339, 0.9365500000000032, 0.8104599999999997, 0.6537666666666655, 0.5040285714285715, 0.39343749999999966, 0.3103888888888887, 0.24968999999999994, 0.07223999999999996, 0.0408600000000001, 0.027724999999999975, 0.02031799999999985, 0.01645666666666669, 0.013148571428571405, 0.01128624999999997, 0.009586666666666625, 0.008474999999999955]
da_list_simu_9 = [5.672999999999856, 6.203425499999861, 6.777343999999894, 7.16783549999996, 7.641531000000006, 8.22963199999996, 8.704002857142816, 9.28528275000002, 9.865346999999968, 10.489377000000033, 17.25281717368432, 24.916865934537473, 31.93834846714585, 37.39170788631537, 41.54234959721159, 45.13303234684957, 48.111577401678346, 51.13718646390864, 53.505465715337266, 67.45158357947722, 70.97925346068722, 72.70638543334549, 73.98591073610562, 74.84455550378766, 75.59048797785557, 76.2668632750841, 76.755374486

ps_list_simu_148 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99996, 0.9935500000000045, 0.8690474999999998, 0.5572599999999991, 0.3019716666666668, 0.15624999999999997, 0.08233000000000001, 0.04337222222222215, 0.023324999999999884]
da_list_simu_148 = [5.672999999999856, 5.690018999999857, 5.7070379999998595, 5.741075999999864, 5.752421999999867, 5.775113999999863, 5.812393714285593, 5.820497999999898, 5.82238899999988, 5.868151199999914, 6.0998932499999965, 6.316129100000008, 6.519411600000019, 6.736233660000025, 6.970131449999992, 7.1927966999999855, 7.402484962500039, 7.652057566666682, 7.906573559999999, 10.684774469131286, 14.375986442177739, 17.638093074818205, 18.588470528632804, 18.40726829379332, 17.986446106067632, 17.688559753784425, 17.460500930005747, 17.32205328607268]
t_list_simu_148 = [0.19999999999999718, 0.39999999999999436, 0.6000000000000113, 0.7999999999999887, 1.0, 1.2000000000000226, 1.4000000000000128, 1.59

ps_list_simu_74 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999777777777777, 0.9998600000000001, 0.8055249999999987, 0.2607233333333335, 0.07164750000000014, 0.020277999999999886, 0.005743333333333329, 0.001661428571428563, 0.0004587499999999971, 0.00012222222222222186, 3.7000000000000025e-05]
da_list_simu_74 = [5.672999999999856, 5.729729999999859, 5.759985999999862, 5.830425749999874, 5.86020899999988, 5.892355999999873, 5.971237714285614, 6.002743124999912, 6.061285333333243, 6.113224799999943, 6.565930200000023, 7.0468115000000004, 7.5418280250000365, 8.072225159999999, 8.629389399999997, 9.213033042857141, 9.791598000000068, 10.425686114393923, 11.100297738979606, 17.835940220099754, 18.251077162749183, 17.62235000350851, 17.376715481392328, 17.227881432650758, 16.864697727272645, 17.13096710526309, 18.062617924528308, 18.398918918918923]
t_list_simu_74 = [0.19999999999999718, 0.39999999999999436, 0.6000000000000113, 0.799999999999988

ps_list_simu_37 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99995, 0.9996200000000001, 0.9948666666666676, 0.9763999999999977, 0.9181875000000007, 0.8251777777777778, 0.693629999999999, 0.0557149999999999, 0.0041733333333333475, 0.00033500000000000023, 2.0000000000000005e-05, 1.6666666666666669e-06, 0.0, 0.0, 0.0, 0.0]
da_list_simu_37 = [5.672999999999856, 5.772277499999858, 5.930175999999868, 6.009125249999888, 6.095071199999907, 6.245027499999898, 6.398333571428481, 6.465092624999965, 6.569964333333286, 6.652159800000002, 7.80264420000004, 9.003429200000012, 10.303339922368487, 11.840259063724474, 13.558942024196774, 15.132859207472263, 16.58499434183273, 17.525227473101737, 18.039201557655918, 17.38897121229356, 16.78389547383302, 16.19261811023622, 16.073500000000003, 28.365000000000002, 28.365000000000002, 28.365000000000002, 28.365000000000002, 28.365000000000002]
t_list_simu_37 = [0.19999999999999718, 0.39999999999999436, 0.6000000000000113, 0.79999999999998

ps_list_simu_18 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9992499999999999, 0.9903333333333328, 0.9148000000000029, 0.7512599999999996, 0.5670666666666675, 0.4178428571428562, 0.3107374999999998, 0.2362888888888888, 0.18576999999999955, 0.04551499999999988, 0.0240266666666667, 0.015717499999999992, 0.010779999999999906, 0.007138333333333354, 0.004989999999999988, 0.0034399999999999973, 0.0022666666666666543, 0.0016079999999999795]
da_list_simu_18 = [5.672999999999856, 6.033235499999851, 6.295138999999864, 6.712577249999915, 6.959636399999968, 7.352207999999981, 7.652876999999885, 7.908162000000039, 8.344982999999969, 8.682526500000046, 12.730309867543896, 17.414509762249196, 21.992381698645413, 25.192765503720718, 27.609688869750084, 29.55558889077823, 31.28616548296212, 32.914733398626765, 34.41176837577587, 43.08639956018718, 44.833257061014834, 45.25608940163005, 45.348019260926485, 45.37228197313191, 45.382060512819386, 45.383999999998885, 45.38399999999893, 45.3839999